# Azure ML Pipeline

Azure ML パイプラインによって、機械学習の個々のタスクをひとつのステップとして扱い、それら全体をひとつの実行単位として扱うことができます。この仕組みによって MLOps のオペレーションを簡素化します。

Azure machine learning pipeline allows each machine learning task to be treated as as step of a pipeline, while managing the entire workflow as a single execution unit. This mechanism simplifies MLOps operations.

<p align="center">
  <img src="./pipeline.png" alt="Pipeline Diagram" width="300"/>
</p>

In [ ]:
# import required libraries
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import dsl, Input, Output, MLClient, load_component
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.constants import AssetTypes

<h5> Azure認証とMLClientの設定</h5>
Azureへの認証を設定し、ワークスペースのMLClientを初期化します。

In [ ]:
try:
    credential = DefaultAzureCredential()
    # Check if given credential can get token successfully.
    credential.get_token("https://management.azure.com/.default")
except Exception as ex:
    # Fall back to InteractiveBrowserCredential in case DefaultAzureCredential not work
    credential = InteractiveBrowserCredential()

# Get a handle to workspace
ml_client = MLClient.from_config(credential=credential)

### COMPONENTS
👉 
##### [Azure Machine Learning コンポーネントとは](https://learn.microsoft.com/ja-jp/azure/machine-learning/concept-component?view=azureml-api-2)

## Load Components

<h5> パイプラインコンポーネントの読み込み</h5>
YAMLファイルで定義されたトレーニング、評価、登録のコンポーネントを読み込みます。

In [ ]:
parent_dir = "component"
train_component = load_component(source=parent_dir + "./train.yml")
evaluate_component = load_component(source=parent_dir + "./evaluate.yml")
register_component = load_component(source=parent_dir + "./register.yml")

##### Sample to Register components for reuse, but not used in this script

<h5> コンポーネントの登録（サンプル）</h5>
再利用のためにコンポーネントをワークスペースに登録するサンプルコードです。

In [ ]:
'''# get or create component
try:
    train_component = ml_client.components.get(
        name=train_component.name, version=train_component.version
    )
except:
    train_component = ml_client.components.create_or_update(train_component)

print(train_component)'''

### PIPELINE
👉 
##### [チュートリアル: 運用環境の機械学習パイプラインを作成する](https://learn.microsoft.com/ja-jp/azure/machine-learning/tutorial-pipeline-python-sdk?view=azureml-api-2)

## Define pipeline

<h5> パイプラインの定義</h5>
トレーニング→評価→登録の順序でコンポーネントを実行するパイプラインを定義します。

In [ ]:
@dsl.pipeline(
    compute="demo-cpucluster1",
    description="diabetes pipeline test"
)
def pipeline_diabetes_01(
    diabetes_data,
    alpha,
    experiment_name,
    model_name
):
    # using data_prep_function like a python call with its own inputs
    train_job = train_component(
        input_data = diabetes_data,
        alpha = alpha
    )
    evaluate_job = evaluate_component(
        experiment_name = experiment_name,
        model_dir = train_job.outputs.output_dir
    )
    register_job = register_component(
        model_name = model_name,
        model_path = train_job.outputs.output_dir,
        deploy_flag = evaluate_job.outputs.deploy_flag
    )  
    # a pipeline returns a dictionary of outputs
    # keys will code for the pipeline output identifier
    return {
        "train_job": train_job.outputs.output_dir,
        "evaluate_job": evaluate_job.outputs.deploy_flag
    }

## Instantiate pipeline

<h5>実験名の設定</h5>
パイプライン実行用の実験名を設定します。

In [ ]:
experiment_name = "diabetes_training_pipeline4"

<h5> パイプラインのインスタンス化</h5>
具体的なパラメータを指定してパイプラインのインスタンスを作成します。

In [ ]:
# Let's instantiate the pipeline with the parameters of our choice
pipeline = pipeline_diabetes_01(
    diabetes_data = Input(type=AssetTypes.URI_FILE, path="diabetes_data_01@latest"),
    alpha = 0.03,
    model_name="diabetes_model_pipeline",
    experiment_name = experiment_name # this name is used for searching MLFlow exp name and need to coincide to pipeline's exp name 
)

## Run pipeline

<h5>パイプラインの実行</h5>
設定されたパイプラインをAzure ML上で実行します。

In [ ]:
# submit the pipeline job
pipeline_job = ml_client.jobs.create_or_update(
    pipeline,
    # Project's name
    experiment_name=experiment_name,
)